In [46]:
import re
import json
import numpy as np
import pandas as pd
import os
from transformers import *
import sys
import pandas as pd
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings('ignore')

In [47]:

def one_hot(idx, length):
    """
    create one-hot vectors
    :param idx: non-zero element
    :param length: size of label space
    :return:
    """
    output = [0] * length
    output[idx] = 1
    return output


# ------------------------------------------------------------------------
# Preprocessing
#-------------------------------------------------------------------------
def replacements(text):
    text = re.sub(r'[@＠][a-zA-Z0-9_.!?-\\]+', "@username", text.rstrip())
    # text = re.sub(r"((www\.[^\s]+)|(https?:\/\/[^\s]+))", "url", text.rstrip())
    text = re.sub(r'https?://|www\.', "url", text.rstrip())
    text = text.replace('-', ' ')
    return text

In [87]:
def posts_to_BERT_feature(data, data_partition):


    b_tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)
    data_folder = '../'
    feature_list = {}
    print(data_partition)
    count = 0
    for ii, row in data.iterrows():
        # print("{}: {}".format(row['ID'], row['Text']))
        if type(row['combined']) != float:
            text = row['combined'].strip('\r\n')

            text = replacements(text)
            if text == ' ':
                text = '-'
            q_tokenized = b_tokenizer.encode(text, add_special_tokens=True,max_length=700 )


            features = {
                'id': row['id'],
                'post':text,
                'tokenized': q_tokenized,
                'label': row['flair'],
                'y': one_hot(label2idx[row['flair']], 12),
            }

            
            feature_list[row['id']] = features
            count += 1
    print(count)
    file_name = os.path.join(data_folder, '{}.json'.format(data_partition))
    json.dump(feature_list, open(file_name, mode='w'))
    print('Preprocessed preprocessed_data dumped for {} set!'.format(data_partition))




In [88]:
df = pd.read_csv('../reddit-india-data.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1118 entries, 0 to 1117
Data columns (total 10 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   flair      1118 non-null   object
 1   title      1118 non-null   object
 2   score      1118 non-null   int64 
 3   id         1118 non-null   object
 4   url        1118 non-null   object
 5   comms_num  1118 non-null   int64 
 6   body       677 non-null    object
 7   author     1118 non-null   object
 8   comments   1019 non-null   object
 9   timestamp  1118 non-null   object
dtypes: int64(2), object(8)
memory usage: 87.5+ KB


In [89]:
df['body'] = df['body'].fillna('')
df['comments'] = df['comments'].fillna('')
df['combined'] = df['title'] + df['body'] + df['comments']

In [90]:
df['flair'].unique()

array(['AskIndia', 'Non-Political', '[R]eddiquette', 'Scheduled',
       'Photography', 'Science/Technology', 'Politics',
       'Business/Finance', 'Policy/Economy', 'Sports', 'Food', 'AMA'],
      dtype=object)

In [91]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1118 entries, 0 to 1117
Data columns (total 11 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   flair      1118 non-null   object
 1   title      1118 non-null   object
 2   score      1118 non-null   int64 
 3   id         1118 non-null   object
 4   url        1118 non-null   object
 5   comms_num  1118 non-null   int64 
 6   body       1118 non-null   object
 7   author     1118 non-null   object
 8   comments   1118 non-null   object
 9   timestamp  1118 non-null   object
 10  combined   1118 non-null   object
dtypes: int64(2), object(9)
memory usage: 96.2+ KB


In [92]:
label2idx = {}
i = 0
for fl in df['flair'].unique():
    label2idx[fl] = i
    i += 1
label2idx

{'AskIndia': 0,
 'Non-Political': 1,
 '[R]eddiquette': 2,
 'Scheduled': 3,
 'Photography': 4,
 'Science/Technology': 5,
 'Politics': 6,
 'Business/Finance': 7,
 'Policy/Economy': 8,
 'Sports': 9,
 'Food': 10,
 'AMA': 11}

In [93]:
df_train, df_test = train_test_split(df,test_size = 0.2)

In [94]:
print(len(df_test))
len(df_train)

224


894

In [95]:
posts_to_BERT_feature(df_train, 'train')
posts_to_BERT_feature(df_test, 'test')
print('done!')

Token indices sequence length is longer than the specified maximum sequence length for this model (1772 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (614 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (553 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (1229 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (3856 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for 

train


Token indices sequence length is longer than the specified maximum sequence length for this model (933 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (529 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (2989 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (1010 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (642 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for t

Token indices sequence length is longer than the specified maximum sequence length for this model (998 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (742 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (678 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (735 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (572 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for thi

Token indices sequence length is longer than the specified maximum sequence length for this model (1232 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (1842 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (2315 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (577 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (850 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for 

...

Token indices sequence length is longer than the specified maximum sequence length for this model (3089 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (548 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (993 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (1047 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (1751 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for 

test


Token indices sequence length is longer than the specified maximum sequence length for this model (818 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (617 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (572 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (1178 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (1224 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for t

Token indices sequence length is longer than the specified maximum sequence length for this model (1074 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (1204 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (1082 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (733 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (2237 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for

224
Preprocessed preprocessed_data dumped for test set!
done!
